# Analysis movies

In [1]:
#import 
%reload_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np
import get_files as gf

# Bonus 2: Récupérer et extraire le jeu de données fourni de manière automatique

In [2]:
url_file ='https://stdatalake005.blob.core.windows.net/public/movies_dataset.zip'
gf.download_file(url_file)

data.zip existe déja
data existe déja


# Chargement des données 


In [3]:

# chagement du fichier name.basics.tsv
df_name_basics= pd.read_csv('../../data/name.basics.tsv', sep ='\t', na_values = ["\\N", "nan"])


# chargement du fichier title.principals.tsv
df_title_principals = pd.read_csv('../../data/title.principals.tsv',sep ='\t', na_values = ["\\N", "nan"])


# chargement du fichier title.basics.tsv
df_title_basics = pd.read_csv('../../data/title.basics.tsv', sep ='\t', na_values = ['\\N', 'nan'])

# chargement du fichier title.akas.tsv
df_title_akas = pd.read_csv('../../data/title.akas.tsv', delimiter ='\t', na_values =["\\N", "nan"])


# chargement du fichier title.ratings.tsv
df_title_ratings= pd.read_csv('../../data/title.ratings.tsv', sep ='\t', na_values = ["\\N", "nan"])




# 1. Liste des Acteurs par Films

In [4]:

df_name_basics = df_name_basics.drop_duplicates()
df_name_basics_filter = df_name_basics.filter(['nconst','primaryName'])

df_title_principals = df_title_principals.drop_duplicates()
df_title_principals_filter = df_title_principals.filter(['tconst','nconst', 'category'])

df_title_principals_actor = df_title_principals_filter.loc[(df_title_principals_filter['category']== 'actor') | (df_title_principals_filter['category']== 'actress')]
df_title_principals_actor = df_title_principals_actor.filter(['tconst','nconst'])

df_title_basics = df_title_basics.drop_duplicates()
df_title_basics_movies = df_title_basics[(df_title_basics.titleType == "movie") | (df_title_basics.titleType == "tvMovie")]
df_title_basics_movies = df_title_basics_movies.filter(['tconst','originalTitle'])

df_merge_actor = pd.merge(df_name_basics_filter,df_title_principals_actor, on = "nconst", how ='inner')
df_merge_actor_movie = pd.merge(df_merge_actor,df_title_basics_movies, on="tconst", how ="inner")
df_merge_actor_movie_filter = df_merge_actor_movie[["originalTitle","primaryName"]]
df_merge_actor_movies = df_merge_actor_movie_filter.groupby(['originalTitle'])['primaryName'].apply(','.join).reset_index()

df_merge_actor_movies.to_csv('../OUT/actor_movies.csv', sep ='\t')

del(df_name_basics_filter,df_title_principals_filter,df_title_principals_actor,df_title_basics_movies,df_merge_actor,df_merge_actor_movie_filter,df_merge_actor_movies)


# 2. Liste des Films Américains 

In [6]:
# get only movies distriuted in US
df_akas_us = df_title_akas.loc[df_title_akas['region'] == "US"].filter(['titleId', 'title', 'region'])
df_akas_us.rename(columns = {'title': 'title_us', 'region': 'region_us'}, inplace = True)

# get only movies distriuted in FR
df_akas_fr = df_title_akas.loc[df_title_akas['region'] == "FR"].filter(['titleId', 'title', 'region'])
df_akas_fr.rename(columns = {'title': 'title_fr', 'region': 'region_fr'}, inplace = True)

# merge movies distributed in US and in FR
df_akas_merged = pd.merge(df_akas_us, df_akas_fr, how='left', on='titleId')

# create new column "is_title_fr" to describe if movie has english title or only french title
df_akas_merged['is_title_fr'] = np.where(df_akas_merged['title_fr'].isnull(), False, True)

# remove US title if FR title exists
df_akas_merged['title_us'] = np.where(df_akas_merged['title_fr'].notnull(), '', df_akas_merged['title_us'])

# merge with rating
df_rating_akas = pd.merge(df_akas_merged, df_title_ratings, how='inner', right_on='tconst', left_on='titleId')

# remove columns
df_rating_akas = df_rating_akas.filter(['titleId', 'title_fr', 'title_us', 'is_title_fr', 'averageRating'])
# generate csv file
df_rating_akas.to_csv('../OUT/american_movies.csv',sep ='\t')

del(df_akas_fr, df_akas_us, df_akas_merged, df_rating_akas)


# 3. Les notes moyennes des différents genres

In [8]:
# split column gere by ','
df_title_basic_filter = df_title_basics[['tconst', 'genres']].dropna()
df_title_basic_genre = pd.DataFrame(df_title_basic_filter.genres.str.split(',').tolist(), index = df_title_basic_filter.tconst).stack()

# define 'tconst' as index
df_title_basic_genre = df_title_basic_genre.reset_index([0, 'tconst'])

# rename columns
df_title_basic_genre.columns = ['tconst', 'genre']

# merge new df with rating dataframe
merge_genre_rating = pd.merge(df_title_basic_genre, df_title_ratings, how = 'inner', on = 'tconst')

# keep only titles 'genre' and 'averageRating'
merge_genre_rating = merge_genre_rating.filter(['genre', 'averageRating'])

# calcul of average by genre
avg_rating_genre = merge_genre_rating.groupby('genre').mean('averageRating')

avg_rating_genre = avg_rating_genre['averageRating'].round(decimals=2)

# avg_rating_genre to csv
avg_rating_genre.to_csv('../OUT/genre_average.csv',sep ='\t')

del(df_title_basic_filter,df_title_basic_genre,merge_genre_rating,avg_rating_genre)

# 4. Note moyenne des acteurs par rapport aux films

In [5]:

df_merge_actor_moy= df_merge_actor_movie[["primaryName","originalTitle","tconst"]]

df_title_ratings_filter = df_title_ratings.filter(['tconst', 'averageRating'])

df_merge_actor_moy_note = pd.merge(df_merge_actor_moy,df_title_ratings_filter, on ='tconst', how = 'inner')
df_merge_actor_moy_note = df_merge_actor_moy_note.groupby(['primaryName'])['averageRating'].mean()

df_merge_actor_moy_note.to_csv('../OUT/actor_average.csv',sep ='\t')





